In [1]:
import numpy as np

# ---------- HMM DEFINITION ----------

states = ["S1", "S2", "S3", "S4"]          # /h/, /e/, /l/, /o/
observations = ["O1", "O2", "O3", "O4"]    # feature vectors

# Transition matrix A (from rows -> to columns)
A = np.array([
    [0.0, 0.7, 0.3, 0.0],   # from S1
    [0.0, 0.2, 0.6, 0.2],   # from S2
    [0.0, 0.0, 0.3, 0.7],   # from S3
    [0.0, 0.0, 0.1, 0.9]    # from S4
])

# Emission matrix B (rows = states, cols = observations O1..O4)
B = np.array([
    [0.6, 0.2, 0.1, 0.1],   # S1 (/h/)
    [0.1, 0.7, 0.1, 0.1],   # S2 (/e/)
    [0.1, 0.1, 0.6, 0.2],   # S3 (/l/)
    [0.2, 0.1, 0.2, 0.5]    # S4 (/o/)
])

# Initial probabilities π
pi = np.array([1.0, 0.0, 0.0, 0.0])

# Observation sequence: [O1, O2, O3, O4]
obs_seq = ["O1", "O2", "O3", "O4"]
obs_index = [observations.index(o) for o in obs_seq]   # -> [0,1,2,3]


# ---------- VITERBI IMPLEMENTATION ----------

def viterbi(pi, A, B, obs_index):
    N = A.shape[0]          # number of states
    T = len(obs_index)      # length of observation sequence

    delta = np.zeros((T, N))   # highest probability of any path that ends in state i at time t
    psi = np.zeros((T, N), dtype=int)  # backpointer

    # 1. Initialization
    delta[0, :] = pi * B[:, obs_index[0]]
    psi[0, :] = 0

    # 2. Recursion
    for t in range(1, T):
        for j in range(N):
            probs = delta[t-1, :] * A[:, j]
            psi[t, j] = np.argmax(probs)
            delta[t, j] = np.max(probs) * B[j, obs_index[t]]

    # 3. Termination
    best_last_state = np.argmax(delta[T-1, :])
    best_path_prob = delta[T-1, best_last_state]

    # 4. Path backtracking
    best_path_states = [best_last_state]
    for t in range(T-1, 0, -1):
        best_state_t = psi[t, best_path_states[-1]]
        best_path_states.append(best_state_t)

    best_path_states = best_path_states[::-1]  # reverse

    # Map to state labels
    best_state_sequence = [states[s] for s in best_path_states]

    return best_state_sequence, best_path_prob, delta, psi


best_seq, best_prob, delta, psi = viterbi(pi, A, B, obs_index)

print("Most likely state sequence:", best_seq)
print("Probability of this sequence:", best_prob)


Most likely state sequence: ['S1', 'S2', 'S3', 'S4']
Probability of this sequence: 0.03704399999999999
